In [44]:
import pandas as pd

In [45]:
df = pd.read_csv('../data/startup_users_visits.csv', parse_dates=['date'])

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373366 entries, 0 to 373365
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   user_id  373366 non-null  object        
 1   date     373366 non-null  datetime64[ns]
 2   pay      373366 non-null  bool          
dtypes: bool(1), datetime64[ns](1), object(1)
memory usage: 6.1+ MB


In [47]:
first_visit = df.groupby('user_id')['date'].min().reset_index()
first_visit.columns = ['user_id', 'first_visit_date']

In [48]:
first_visit

,user_id,first_visit_date
0,0001599900972add,2023-06-15
1,00019b04f17ade68,2022-12-18
2,0001c405d5a77893,2023-07-09
3,000314ab1fe38c66,2023-04-22
4,0003f0dbb8dc8b65,2023-02-17
...,...,...
25643,7ffa82acbb882941,2023-06-03
25644,7ffb5e73d6d7f498,2023-06-02
25645,7ffd24ca0df1f24c,2023-09-08
25646,7ffe53ad76ee5e74,2023-07-26


In [49]:
merged = df.merge(first_visit, on='user_id')
merged['days_since_first'] = (merged['date'] - merged['first_visit_date']).dt.days
within_90 = merged[(merged['days_since_first'] >= 0) & (merged['days_since_first'] < 90)]
within_90

,user_id,date,pay,first_visit_date,days_since_first
0,0001599900972add,2023-06-15,False,2023-06-15,0
1,0001599900972add,2023-06-16,False,2023-06-15,1
2,00019b04f17ade68,2022-12-18,False,2022-12-18,0
3,0001c405d5a77893,2023-07-09,False,2023-07-09,0
4,000314ab1fe38c66,2023-05-12,False,2023-04-22,20
...,...,...,...,...,...
373361,7fff855159083854,2023-09-25,False,2023-09-03,22
373362,7fff855159083854,2023-09-26,False,2023-09-03,23
373363,7fff855159083854,2023-09-28,False,2023-09-03,25
373364,7fff855159083854,2023-09-03,False,2023-09-03,0


In [50]:
paid_users = within_90.groupby('user_id')['pay'].max().reset_index()
paid_users = paid_users[paid_users['pay']]

In [51]:
paid_first_visit = paid_users.merge(first_visit, on='user_id')
paid_first_visit['month'] = paid_first_visit['first_visit_date'].dt.to_period('M').dt.to_timestamp()

In [52]:
result = paid_first_visit.groupby('month').size().reset_index(name='counts')

In [53]:
min_month = first_visit['first_visit_date'].min().to_period('M').to_timestamp() if not first_visit.empty else pd.Timestamp('2023-06-01')
max_month = pd.Timestamp('2023-06-01')
all_months = pd.date_range(start=min_month, end=max_month, freq='MS')
all_months_df = pd.DataFrame({'month': all_months})

In [ ]:

final_result = all_months_df.merge(result, on='month', how='left').fillna(0)
final_result = final_result[final_result['month'] <= max_month].sort_values('month')
final_result

,month,counts
0,2022-11-01,133
1,2022-12-01,552
2,2023-01-01,437
3,2023-02-01,949
4,2023-03-01,950
5,2023-04-01,678
6,2023-05-01,954
7,2023-06-01,848


: 